In [2]:
import pandas as pd

In [3]:
fn = "2022-10-18T09:06:40.464Z_discoveries_01_LSST_TEST.psv"

In [4]:
fn

'2022-10-18T09:06:40.464Z_discoveries_01_LSST_TEST.psv'

In [5]:
def read_psv_ades(fn):
    header = []
    with open(fn) as fp:
        # consume and store the header
        while True:
            line = fp.readline()
            if line[0] in ['#', '!']:
                header.append(line.rstrip())
                continue
            # the line is the header
            names = [ s.strip() for s in line.split('|') ]
            break
        df = pd.read_csv(fp, sep='|', header=0, names=names)
    return df, header

df, hdr = read_psv_ades(fn)
df

,trkSub,obsTime,ra,dec,rmsRA,rmsDec,mag,rmsMag,band,stn,mode,astCat
0,t0000000,2022-10-04T04:31:35.818Z,340.215384,-31.237961,0.024390,0.028525,21.200,0.068814,y,I11,CCD,Gaia2
1,t0000000,2022-10-08T04:57:44.150Z,339.602568,-31.208910,0.030950,0.036187,21.247,0.092267,y,I11,CCD,Gaia2
2,t0000000,2022-10-08T05:20:55.363Z,339.600181,-31.208704,0.031220,0.036502,21.253,0.074054,z,I11,CCD,Gaia2
3,t0000000,2022-10-08T05:42:49.939Z,339.597960,-31.208498,0.039912,0.046665,21.253,0.085480,z,I11,CCD,Gaia2
4,t0000000,2022-10-08T06:06:00.720Z,339.595550,-31.208269,0.049422,0.057784,21.248,0.124832,y,I11,CCD,Gaia2
...,...,...,...,...,...,...,...,...,...,...,...,...
455044,t0030307,2022-10-17T01:27:39.254Z,341.016670,-13.647847,0.107355,0.110474,22.814,0.136485,i,I11,CCD,Gaia2
455045,t0030307,2022-10-18T00:39:11.635Z,340.963762,-13.645669,0.030286,0.031165,22.952,0.064220,r,I11,CCD,Gaia2
455046,t0030307,2022-10-18T00:39:50.256Z,340.963714,-13.645656,0.029608,0.030468,22.952,0.063283,r,I11,CCD,Gaia2
455047,t0030307,2022-10-18T01:02:39.437Z,340.962809,-13.645616,0.050079,0.051533,23.427,0.082563,g,I11,CCD,Gaia2


In [6]:
hdr

['# version=2017',
 '# observatory',
 '! mpcCode I11',
 '# submitter',
 '! name D. iRAC',
 '# telescope',
 '! aperture 8.4',
 '! design Reflector',
 '! detector CCD',
 '# observers',
 '! name D. iRAC',
 '# measurers',
 '! name D. iRAC',
 '# comment',
 '! line MPC-Rubin Observatory Integration Tests']

In [7]:
del df["rmsMag"]

In [8]:
def write_psv_ades(fn, df, header):
        with open(fn, "w") as fp:
            fp.write('\n'.join(header))
            fp.write('\n')
            df.to_csv(fp, sep='|', index=False)

t1 = df[df["trkSub"] == 't0000000']
write_psv_ades("foo.psv", t1, hdr)

In [105]:
def fit_orbit(df, hdr):
    # create a temporary directory
    import tempfile, subprocess, json, os
    with tempfile.TemporaryDirectory() as tmpdir:
        print(f"tmpdir: {tmpdir}")

        # prep the new "home" directory
        from shutil import copytree, ignore_patterns
        copytree(
            os.path.expanduser("~/.find_orb"), f"{tmpdir}/.find_orb",
            ignore=ignore_patterns('debug.txt', 'elements.json', 'elem_short.json', 'linux_p1550p2650.430t')
        )
        os.symlink(os.path.expanduser("~/.find_orb/linux_p1550p2650.430t"), f"{tmpdir}/.find_orb/linux_p1550p2650.430t")

        # dump to ades
        datafile = f"{tmpdir}/data.psv"
        resultdir = f"{tmpdir}/result"
        write_psv_ades(datafile, df, hdr)

        # call findorb
        cmd = f"fo {datafile} -O {resultdir} -D /Users/mjuric/projects/elasticsky/environ.dat"
        env = os.environ.copy()
        env["HOME"] = tmpdir
        ret = subprocess.run(cmd, shell=True, env=env, check=False, capture_output=True)

        # fetch/construct the result
        if ret.returncode == 0:
            # read the result
            with open(f"{resultdir}/covar.json") as fp:
                result = json.load(fp)
        else:
            result = {}

        result["name"] = df["trkSub"].iloc[0]
        result["findorb"] = {
            'args': ret.args,
            'returncode': ret.returncode,
            'stdout': ret.stdout.decode('utf-8'),
            'stderr': ret.stderr.decode('utf-8')
        }

        return result

orbit = fit_orbit(t1, hdr)
orbit

tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp9ect2eh_


{'covar': [[4.86890498341e-08,
   -2.85905193023e-08,
   -2.18704663892e-08,
   5.18315230761e-10,
   -4.42856419266e-10,
   -1.78272223151e-10],
  [-2.85905193023e-08,
   1.67887487008e-08,
   1.28424628164e-08,
   -3.03478040476e-10,
   2.59486489349e-10,
   1.04298059729e-10],
  [-2.18704663892e-08,
   1.28424628164e-08,
   9.82403935329e-09,
   -2.33260592426e-10,
   1.9919678254e-10,
   8.02585044027e-11],
  [5.18315230761e-10,
   -3.03478040476e-10,
   -2.33260592426e-10,
   1.92722397751e-11,
   -1.32440110674e-11,
   -7.88562465198e-12],
  [-4.42856419266e-10,
   2.59486489349e-10,
   1.9919678254e-10,
   -1.32440110674e-11,
   9.31941877889e-12,
   5.3349086317e-12],
  [-1.78272223151e-10,
   1.04298059729e-10,
   8.02585044026e-11,
   -7.88562465198e-12,
   5.3349086317e-12,
   3.26072043199e-12]],
 'state_vect': [3.7322002612575713,
  -1.4273427590100565,
  -1.232355442812986,
  0.001063973021473759,
  0.005572386885645831,
  -9.169745980840394e-05],
 'epoch': 2459856.689421

In [106]:
def chunk_submission(df, key, n=10):
    k = df[key].unique()
    chunks = [ k[i:i + n] for i in range(0, len(k), n) ]
    return chunks
tracks = chunk_submission(df, 'trkSub')
print(len(tracks))
tracks[-1][-10:]

3031


array(['t0030300', 't0030301', 't0030302', 't0030303', 't0030304',
       't0030305', 't0030306', 't0030307'], dtype=object)

In [107]:
%%time
def fit_orbit_batch(df, hdr, tracks=None):
    if tracks is None:
        tracks = df['trkSub'].unique()

    results = {}
    for trkSub in tracks:
        trk = df[df["trkSub"] == trkSub]
        results[trkSub] = fit_orbit(trk, hdr)
    return results

orbits = fit_orbit_batch(df, hdr, tracks[0])

tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpw3u0hh_k
tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp7kya8v_d
tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp0llb_hsq
tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmptazd06pn
tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpmwi00nar
tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp08tmd7gs
tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpdvojwb0y
tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpeo3f2xrf
tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp97bge10n
tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmplol9xl4_
CPU times: user 418 ms, sys: 297 ms, total: 715 ms
Wall time: 7.21 s


In [108]:
[ result['findorb']['returncode'] for result in orbits.values() ]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [109]:
[ result['name'] for result in orbits.values() ]

['t0000000',
 't0000001',
 't0000002',
 't0000003',
 't0000004',
 't0000005',
 't0000006',
 't0000007',
 't0000008',
 't0000009']

In [15]:
# parallel execution with Ray
import ray
ray.init()

2020-12-09 15:37:02,398	INFO services.py:1090 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '10.0.1.10',
 'raylet_ip_address': '10.0.1.10',
 'redis_address': '10.0.1.10:6379',
 'object_store_address': '/tmp/ray/session_2020-12-09_15-37-01_808872_10157/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-12-09_15-37-01_808872_10157/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2020-12-09_15-37-01_808872_10157',
 'metrics_export_port': 57603,
 'node_id': 'fb57b83b906600e07a0303c7d7297cd3224382ae'}

In [110]:
@ray.remote
def dist_fit_orbit_batch(df, hdr, tracks):
    return fit_orbit_batch(df, hdr, tracks)

In [143]:
%%time
df_id = ray.put(df)
futures = [
    dist_fit_orbit_batch.remote(df_id, hdr, track_batch)
    for track_batch in tracks[:100]
]
chunked_results = ray.get(futures)
del df_id

(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpqit2zvic
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpapl38ad2
(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpwryy6rpi
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpw3rou9ej
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp9cd8dyk3
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp_dwv_hpi
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpjeqlepc5
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp42tgqr5k
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmputvu6iqb
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpee22zt4l
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp9s2utvrg
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp2hcpjhor
(pid=10945) tmpdir: /var/fol

(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpvosvyshk
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpl3rftd71
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpef6s1ryj
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpdsfdu8_w
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmphjc7xl7a
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpykpsoo0e
(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp4zvvox23
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpnnm30mek
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp9y0t0uvw
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpmzrvacww
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpss8y7upi
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpjjjoe1ej
(pid=10944) tmpdir: /var/fol

(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpyt20ub8z
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpmky3mmzi
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpba4u3qol
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpne3rom27
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpt522yist
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpq64lloe9
(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpt_z44htw
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmprjy9qex5
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpe16k0s5d
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp96wu7bny
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpdbd61a30
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpglqelens
(pid=10945) tmpdir: /var/fol

(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp3vmmabd8
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpzy09wij4
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp8le0v4da
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpbij8zyur
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpbr0kaz1w
(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpuzezr6eq
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpr9gtrtg9
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpt9ci2t73
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmph_3m1cmb
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp7zyp2j1h
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp2dgfwajq
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp9qr5g22p
(pid=10944) tmpdir: /var/fol

(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmppko9t6rk
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpdj184vpi
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp9ne_8svx
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp1lhrzo34
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp4tye_v37
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp320p_8im
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmptems0rn8
(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp9o0cc9aj
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpl545dgon
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmph3at006q
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp_o4e9ni9
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpezley8ea
(pid=10941) tmpdir: /var/fol

...

(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpcbg6pt4x
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmplkcz4019
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp710x_tho
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpuf1h780z
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp3_6u2rak
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpio4p73mf
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp79monu5d
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpw6n23rti
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp603h65cb
(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmph21g3cul
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpmzczkbkf
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp5rr9z4qz
(pid=10945) tmpdir: /var/fol

(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpsdmo19eo
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpcygh2hzl
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpmvzykqm2
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmptpw8hojp
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpz6ck5p74
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmplsdv1p7k
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp4ghsscpq
(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp3lxojmrh
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmplu00l1rw
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpqa3u6e23
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpnl0m0o64
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpvr6k9o2u
(pid=10945) tmpdir: /var/fol

(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpe_i8v7se
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp9h2ks71i
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp1z9wbhl3
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpjkfthnqt
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp4ih7fhn9
(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpwczf6icu
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpdyoy8dne
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpj485zz_n
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpt3zxa6m4
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmplf2pdpqk
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp7jxz0pac
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpkene8bmv
(pid=10943) tmpdir: /var/fol

(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpk1dov_0k
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpbeyfjih1
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpd0obzmd_
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp69dkbfek
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmph5rsfcxk
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpewfltwb9
(pid=10945) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpz3vlfkte
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpha1gvfxg
(pid=10942) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpgug22_2t
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmplp7kzyll
(pid=10941) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpkgc_iu7j
(pid=10943) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp_wksetxt
(pid=10948) tmpdir: /var/fol

(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp4o8f009w
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp26xt66vz
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp5_00mmts
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmptuon_027
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpwrm1w6p4
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpzkmmk8u3
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpq5v3_jof
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpy5jjicm5
(pid=10948) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpkiqaxdna
(pid=10947) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmp3wtsx9ep
(pid=10946) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpgch03yyu
(pid=10944) tmpdir: /var/folders/0n/3cchn_wj5619fqz4dswygvj80000gn/T/tmpplujgero
(pid=10948) tmpdir: /var/fol

In [144]:
orbits = {}
for d in chunked_results:
    orbits.update(d)
returncodes = [ result['findorb']['returncode'] for result in orbits.values() ]
print(returncodes)
#[ result['state_vect'][0] for result in orbits.values() ]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [145]:
len(returncodes)

1000

In [146]:
failures = [ result for result in orbits.values() if result['findorb']['returncode'] != 0]
print(len(failures))
failures
#[ result['state_vect'][0] for result in orbits.values() ]

0


[]

In [147]:
states = [ result['state_vect'] for result in orbits.values() ]
states = pd.DataFrame(states, index=orbits.keys())
states

,0,1,2,3,4,5
t0000000,3.732200,-1.427343,-1.232355,0.001064,0.005572,-0.000092
t0000001,4.013681,0.824397,-1.523476,-0.000825,0.002680,-0.000553
t0000002,2.375318,-0.443731,0.379375,0.001031,0.008871,0.001200
t0000003,1.253650,-0.359243,-0.265275,0.009259,0.013286,-0.002807
t0000004,1.620572,-0.258156,0.222494,0.014663,0.008725,-0.000178
...,...,...,...,...,...,...
t0000995,2.698851,-0.858234,-0.679425,0.004186,0.009364,0.001089
t0000996,2.476280,-1.089072,-0.078946,0.002968,0.009418,0.002253
t0000997,2.263434,-0.065694,0.050989,-0.000750,0.011471,0.000591
t0000998,2.432891,-1.003425,-0.203119,0.004086,0.009957,-0.000663


In [148]:
(states1 == states).all()

0    True
1    True
2    True
3    True
4    True
5    True
dtype: bool